# Qdrantを使用したエンベディング検索

このノートブックでは、データをダウンロードし、エンベディングを作成し、ベクトルデータベースを使用してインデックス化と検索を行う簡単なフローを説明します。これは、チャットボット、トピックモデリングなどの本番環境のユースケースをサポートするために、安全な環境で独自のデータと共にエンベディングを保存・検索したいお客様にとって一般的な要件です。

### ベクトルデータベースとは

ベクトルデータベースは、エンベディングベクトルを保存、管理、検索するために作られたデータベースです。非構造化データ（テキスト、音声、動画など）をベクトルとしてエンコードし、機械学習モデルで利用するエンベディングの使用は、自然言語、画像認識、その他の非構造化データ形式に関わるユースケースでのAIの有効性が高まったことにより、近年爆発的に増加しています。ベクトルデータベースは、企業がこれらのユースケースを提供し、スケールするための効果的なソリューションとして登場しました。

### なぜベクトルデータベースを使用するのか

ベクトルデータベースにより、企業はこのリポジトリで共有している多くのエンベディングユースケース（質問応答、チャットボット、推薦サービスなど）を、安全でスケーラブルな環境で活用できるようになります。多くのお客様は小規模でエンベディングを使って問題を解決していますが、パフォーマンスとセキュリティの問題により本番環境への移行が阻まれています。私たちはベクトルデータベースをその解決の重要な要素と考えており、このガイドではテキストデータのエンベディング、ベクトルデータベースへの保存、セマンティック検索での使用の基本について説明します。

### デモフロー
デモフローは以下の通りです：
- **セットアップ**: パッケージをインポートし、必要な変数を設定
- **データ読み込み**: データセットを読み込み、OpenAIエンベディングを使用してエンベディング化
- **Qdrant**
    - *セットアップ*: QdrantのPythonクライアントをセットアップします。詳細は[こちら](https://github.com/qdrant/qdrant_client)をご覧ください
    - *データのインデックス化*: __titles__と__content__のベクトルを含むコレクションを作成
    - *データ検索*: 動作確認のためにいくつかの検索を実行

このノートブックを実行すると、ベクトルデータベースのセットアップと使用方法の基本的な理解が得られ、エンベディングを活用したより複雑なユースケースに進むことができます。

## セットアップ

必要なライブラリをインポートし、使用したい埋め込みモデルを設定します。

In [ ]:
# We'll need to install Qdrant client
!pip install qdrant-client

In [2]:
import openai
import pandas as pd
from ast import literal_eval
import qdrant_client # Qdrant's client library for Python

# This can be changed to the embedding model of your choice. Make sure its the same model that is used for generating embeddings
EMBEDDING_MODEL = "text-embedding-ada-002"

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## データの読み込み

このセクションでは、このセッションの前に準備した埋め込みデータを読み込みます。

In [3]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

100% [......................................................................] 698933052 / 698933052

'vector_database_wikipedia_articles_embedded (10).zip'

In [4]:
import zipfile
with zipfile.ZipFile("vector_database_wikipedia_articles_embedded.zip","r") as zip_ref:
    zip_ref.extractall("../data")

In [5]:
article_df = pd.read_csv('../data/vector_database_wikipedia_articles_embedded.csv')

In [6]:
article_df.head()

,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


In [7]:
# Read vectors from strings back into a list
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# Set vector_id to be a string
article_df['vector_id'] = article_df['vector_id'].apply(str)

In [8]:
article_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   url             25000 non-null  object
 2   title           25000 non-null  object
 3   text            25000 non-null  object
 4   title_vector    25000 non-null  object
 5   content_vector  25000 non-null  object
 6   vector_id       25000 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


## Qdrant

**[Qdrant](https://qdrant.tech/)**は、Rustで書かれた高性能なベクトル検索データベースです。オンプレミス版とクラウド版の両方を提供していますが、この例では、ローカルデプロイメントモードを使用します。

すべてを設定するには以下が必要です：
- Qdrantのローカルインスタンスを起動する
- コレクションを設定し、データを格納する
- いくつかのクエリで試してみる

### セットアップ

ローカルデプロイメントでは、Qdrantのドキュメントに従ってDockerを使用します：https://qdrant.tech/documentation/quick_start/。Qdrantは単一のコンテナのみを必要としますが、docker-compose.yamlファイルの例がこのリポジトリの`./qdrant/docker-compose.yaml`で利用できます。

このディレクトリに移動して`docker-compose up -d`を実行することで、Qdrantインスタンスをローカルで起動できます。

> Dockerのメモリ制限を8GB以上に増やす必要があるかもしれません。そうしないと、Qdrantが`7 Killed`のようなエラーメッセージで実行に失敗する可能性があります。

In [9]:
! docker compose up -d

[+] Running 1/0
 ✔ Container qdrant-qdrant-1  Running                                      0.0s 


In [10]:
qdrant = qdrant_client.QdrantClient(host="localhost", port=6333)

In [11]:
qdrant.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='Articles')])

### データのインデックス化

Qdrantは__コレクション__にデータを保存し、各オブジェクトは少なくとも1つのベクトルで記述され、__ペイロード__と呼ばれる追加のメタデータを含む場合があります。私たちのコレクションは**Articles**と呼ばれ、各オブジェクトは**title**と**content**の両方のベクトルで記述されます。

すべてのユーティリティメソッドが組み込まれている公式の[qdrant-client](https://github.com/qdrant/qdrant_client)パッケージを使用します。

In [12]:
from qdrant_client.http import models as rest

In [13]:
# Get the vector size from the first row to set up the collection
vector_size = len(article_df['content_vector'][0])

# Set up the collection with the vector configuration. You need to declare the vector size and distance metric for the collection. Distance metric enables vector database to index and search vectors efficiently.
qdrant.recreate_collection(
    collection_name='Articles',
    vectors_config={
        'title': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
        'content': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
    }
)

True

In [14]:
vector_size = len(article_df['content_vector'][0])

qdrant.recreate_collection(
    collection_name='Articles',
    vectors_config={
        'title': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
        'content': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
    }
)

True

`vector`で定義されたベクトル設定に加えて、`payload`設定も定義することができます。Payloadはオプションフィールドで、ベクトルと一緒に追加のメタデータを保存することができます。今回のケースでは、記事の`id`、`title`、`url`を保存します。検索結果でpayloadから最も近い記事のタイトルを返すため、ユーザーに記事のURL（メタデータの一部）も提供することができます。

In [15]:
from qdrant_client.models import PointStruct # Import the PointStruct to store the vector and payload
from tqdm import tqdm # Library to show the progress bar 

# Populate collection with vectors using tqdm to show progress
for k, v in tqdm(article_df.iterrows(), desc="Upserting articles", total=len(article_df)):
    try:
        qdrant.upsert(
            collection_name='Articles',
            points=[
                PointStruct(
                    id=k,
                    vector={'title': v['title_vector'], 
                            'content': v['content_vector']},
                    payload={
                        'id': v['id'],
                        'title': v['title'],
                        'url': v['url']
                    }
                )
            ]
        )
    except Exception as e:
        print(f"Failed to upsert row {k}: {v}")
        print(f"Exception: {e}")

Upserting articles: 100%|█████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [02:52<00:00, 144.82it/s]


In [16]:
# Check the collection size to make sure all the points have been stored
qdrant.count(collection_name='Articles')

CountResult(count=25000)

### データの検索

データがQdrantに格納されたら、最も近いベクトルを求めてコレクションにクエリを開始します。タイトルベースの検索からコンテンツベースの検索に切り替えるために、追加のパラメータ`vector_name`を提供することができます。ファイル内の元の埋め込みがこのモデルで作成されているため、text-embedding-ada-002モデルを使用することを確認してください。

In [17]:
def query_qdrant(query, collection_name, vector_name='title', top_k=20):

    # Creates embedding vector from user query
    embedded_query = openai.embeddings.create(
        input=query,
        model=EMBEDDING_MODEL,
    ).data[0].embedding # We take the first embedding from the list
    
    query_results = qdrant.search(
        collection_name=collection_name,
        query_vector=(
            vector_name, embedded_query
        ),
        limit=top_k, 
        query_filter=None
    )
    
    return query_results

In [18]:
query_results = query_qdrant('modern art in Europe', 'Articles', 'title')
for i, article in enumerate(query_results):
    print(f'{i + 1}. {article.payload["title"]}, URL: {article.payload["url"]} (Score: {round(article.score, 3)})')

1. Museum of Modern Art, URL: https://simple.wikipedia.org/wiki/Museum%20of%20Modern%20Art (Score: 0.875)
2. Western Europe, URL: https://simple.wikipedia.org/wiki/Western%20Europe (Score: 0.867)
3. Renaissance art, URL: https://simple.wikipedia.org/wiki/Renaissance%20art (Score: 0.864)
4. Pop art, URL: https://simple.wikipedia.org/wiki/Pop%20art (Score: 0.86)
5. Northern Europe, URL: https://simple.wikipedia.org/wiki/Northern%20Europe (Score: 0.855)
6. Hellenistic art, URL: https://simple.wikipedia.org/wiki/Hellenistic%20art (Score: 0.853)
7. Modernist literature, URL: https://simple.wikipedia.org/wiki/Modernist%20literature (Score: 0.847)
8. Art film, URL: https://simple.wikipedia.org/wiki/Art%20film (Score: 0.843)
9. Central Europe, URL: https://simple.wikipedia.org/wiki/Central%20Europe (Score: 0.843)
10. European, URL: https://simple.wikipedia.org/wiki/European (Score: 0.841)
11. Art, URL: https://simple.wikipedia.org/wiki/Art (Score: 0.841)
12. Byzantine art, URL: https://simple.

In [19]:
# This time we'll query using content vector
query_results = query_qdrant('Famous battles in Scottish history', 'Articles', 'content')
for i, article in enumerate(query_results):
    print(f'{i + 1}. {article.payload["title"]}, URL: {article.payload["url"]} (Score: {round(article.score, 3)})')

1. Battle of Bannockburn, URL: https://simple.wikipedia.org/wiki/Battle%20of%20Bannockburn (Score: 0.869)
2. Wars of Scottish Independence, URL: https://simple.wikipedia.org/wiki/Wars%20of%20Scottish%20Independence (Score: 0.861)
3. 1651, URL: https://simple.wikipedia.org/wiki/1651 (Score: 0.852)
4. First War of Scottish Independence, URL: https://simple.wikipedia.org/wiki/First%20War%20of%20Scottish%20Independence (Score: 0.85)
5. Robert I of Scotland, URL: https://simple.wikipedia.org/wiki/Robert%20I%20of%20Scotland (Score: 0.846)
6. 841, URL: https://simple.wikipedia.org/wiki/841 (Score: 0.844)
7. 1716, URL: https://simple.wikipedia.org/wiki/1716 (Score: 0.844)
8. 1314, URL: https://simple.wikipedia.org/wiki/1314 (Score: 0.837)
9. 1263, URL: https://simple.wikipedia.org/wiki/1263 (Score: 0.836)
10. William Wallace, URL: https://simple.wikipedia.org/wiki/William%20Wallace (Score: 0.835)
11. Stirling, URL: https://simple.wikipedia.org/wiki/Stirling (Score: 0.831)
12. 1306, URL: https: